# Introduction
This Jupyter Notebook automates the comparison analysis when CMS releases their new PFS-Relative-Value-Files. Upon the execution, it will perform the following actions in order:
> 1. Download the PFS-Relative-Value-Files of the current year and the previous year from CMS website to your current directory.
> 2. Generate a histogram plot to show the number of CPT codes that have a value change in the work RVU, by their work RVU difference and save the image to your current directory.
> 3. Generate an excel file with a listing of CPT codes that have a value change in the work RVU, work RVU in both years, and the work RVU difference sorting in descending order by the difference. The histogram plot graph will also be saved in the same excel file.

## Prerequisites
> Recommend an installation of Anaconda before running the scripts below. Otherwise, you may need to use pip commands to install the following packages in your environment.


In [ ]:
%matplotlib inline
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
sns.set(style="darkgrid")
from bs4 import BeautifulSoup, SoupStrainer
import requests, zipfile, io, datetime, os


<h3 id="Dl"> Download PFS-Relative-Value-Files from CMS website </h3>


The RVU analysis is defaulted to compare current year(cy) RVU values with the previous year(py)'s. It is defaulted to download the first released file in the current year and the previous year respectively. If you would like to change the current year to another year, remove the command lines and enter the year values. 

In [ ]:

cdate=datetime.date.today()
cy=cdate.strftime("%Y")
cqr='a'

py=cdate.year-1
py=str(py)
pqr=cqr

'''
#remove this for customized settings on timeframe.
cy='2021'  #in YYYY format: 2021,2020, etc.
py='2020'  #in YYYY format: 2021,2020, etc.
cqr='d'  #a: January Release b: April Release c: July Release d: October Release
pqr='d' #a: January Release b: April Release c: July Release d: October Release
'''

def strmonth(qr):
    #find the month key for the input value.
    if qr=='a':
        month='JAN'
    elif qr=='b':
        month='APR'
    elif qr=='c':
        month='JUL'
    elif qr=='d':
        month='OCT'
    return month
            
cmonth=strmonth(cqr)
pmonth=strmonth(pqr)

print(cy[-2:])
print(cmonth)
print(py[-2:])
print(pmonth)

In [ ]:
curl='https://www.cms.gov/medicaremedicare-fee-service-paymentphysicianfeeschedpfs-relative-value-files/rvu' + cy[-2:] + cqr
purl='https://www.cms.gov/medicaremedicare-fee-service-paymentphysicianfeeschedpfs-relative-value-files/rvu' + py[-2:] + pqr

print(curl)
print(purl)

Retrive the download links from the website, download and extract the zip files to the current directory.

In [ ]:
def download_link(url):
    #return the file download link from the CMS Website
    page = requests.get(url)    
    data = page.text
    soup = BeautifulSoup(data)
    for link in soup.find_all('a'):
        if "files/zip" in link.get('href'):
            ziplink=link.get('href')
            url_zip='https://www.cms.gov'+ziplink
    return url_zip
print(download_link(curl))
print(download_link(purl))

In [ ]:
def zipextract(url_zip):
    #download zip files and extract all to the current directory
    response = requests.get(url_zip)
    zipDocument = zipfile.ZipFile(io.BytesIO(response.content))
    zipDocument.extractall()

zipextract(download_link(curl))
zipextract(download_link(purl))


If you are not sure where the files are saved at (where the current directory is), run the code below to find out.

In [ ]:
CURR_DIR = os.getcwd()
print(CURR_DIR)

<h3 id="Cl"> Data Cleanup</h3>

Load two RVU files into two data sets, perform a quality check and clean up the data.

In [ ]:
def fileloc(yr,mon):    
   # return filepath of the downloaded files
    filepath=CURR_DIR+'\PPRRVU'+yr+'_'+mon+'.csv'
    return filepath

cdf=pd.read_csv(fileloc(cy[-2:],cmonth),skiprows=9)
pdf=pd.read_csv(fileloc(py[-2:],pmonth), skiprows=9)

In [ ]:
def df_clean(df):
    #assign column names, select needed columns, EOF marker check, and remove rows with modifiers
    df.columns=['HCPCS','MOD','DESCRIPTION','STATUS_CODE','NOT_USED_FOR_MEDICARE _PAYMENT','WORK_RVU','NON-FAC_PE RVU','NON-FAC_NA_INDICATOR','FACILITY_PE RVU','FACILITY_NA_INDICATOR','MP_RVU','NON-FACILITY_TOTAL','FACILITY_TOTAL','PCTC_IND','GLOB_DAYS','PRE_OP','INTRA_OP','POST_OP','MULT_PROC','BILAT_SURG','ASST_SURG','CO-_SURG','TEAM_SURG','ENDO_BASE','CONV_FACTOR','PHYSICIAN_SUPERVISION_DIAGNOSTIC_PROCEDURES','CALCULATION_FLAG','DIAGNOSTIC_IMAGING_FAMILY_INDICATOR','PE_USED_PAYMENT_AMOUNT','PE_USED_PAYMENT_AMOUNT','MP_USED_PAYMENT_AMOUNT']
    df = df[['HCPCS','MOD', 'DESCRIPTION', 'WORK_RVU', 'CONV_FACTOR']]

    if df.iloc[-1,0]=='\x1a': #cheack for an end of a file (EOF) marker
        df=df[:-1]
    else: print(df,'file is lack of an EOF marker, please check for its completeness')

    #drop rows with modifiers
    df=df.loc[df['MOD'].isnull()]
    return df


cdf_clean=df_clean(cdf)
pdf_clean=df_clean(pdf)
print(cdf_clean)
print(pdf_clean)


In [ ]:
'''
#Quality Check
cdf_clean.dtypes
cdf_clean.info
pdf_clean.dtypes
pdf_clean.info

'''

join_df=pd.merge(cdf_clean,pdf_clean,on="HCPCS",how="outer",suffixes=('_cy', '_py'))
join_df=join_df.assign(RVU_DIFF=join_df['WORK_RVU_cy']-join_df['WORK_RVU_py'])
rvu_diff_df=join_df.loc[(join_df['RVU_DIFF']!=0.0) & (join_df['RVU_DIFF'].notnull())]
rvu_diff_df

In [ ]:
plt.hist(rvu_diff_df['RVU_DIFF'],bins=10)
plt.ylabel('Count')
plt.xlabel('RVU Difference')
fig1 = plt.gcf()
plt.show()

<h3 id="Op">Output File Generation</h3>

Generate an excel file with a listing of CPT codes and save the histogram plot into the same file.

In [ ]:
#sort df by RVU_DIFF values
rvu_diff_df=rvu_diff_df.sort_values(by='RVU_DIFF', ascending=False)

#save the histogram into excel
fig1.savefig(pmonth+py+'_'+cmonth+cy+'_RVU_Difference_Histogram.png')


In [ ]:
# Create a Pandas Excel writer using XlsxWriter as the engine.
writer = pd.ExcelWriter(CURR_DIR +'\\'+ pmonth+py+'-'+cmonth+cy+'_RVU_Difference.xlsx', engine='xlsxwriter')

# Convert the dataframe to an XlsxWriter Excel object.
rvu_diff_df.to_excel(writer, sheet_name='Data', index=False)

# Get the xlsxwriter workbook and worksheet objects.
workbook  = writer.book
worksheet = writer.sheets['Data']

# Insert an image.
worksheet.insert_image('M3',CURR_DIR+'\\'+pmonth+py+'_'+cmonth+cy+'_RVU_Difference_Histogram.png')

# Close the Pandas Excel writer and output the Excel file.
writer.save()